### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [70]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 200)


In [89]:
# master_user = pd.read_csv('365-report/users_4_7_2022 2 48 41 AM.csv')

master_user = pd.read_csv('365-report/users_9_12_2022 6 14 26 AM.csv')
master_user.columns = map(str.lower, master_user.columns)

In [90]:
master_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7811 entries, 0 to 7810
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   block credential                 7811 non-null   bool   
 1   city                             6102 non-null   object 
 2   country/region                   214 non-null    object 
 3   department                       5737 non-null   object 
 4   dirsyncenabled                   7304 non-null   object 
 5   display name                     7811 non-null   object 
 6   fax                              15 non-null     object 
 7   first name                       7459 non-null   object 
 8   last dirsync time                7304 non-null   object 
 9   last name                        7276 non-null   object 
 10  last password change time stamp  7811 non-null   object 
 11  license assignment details       0 non-null      float64
 12  licenses            

In [91]:
# soft deletion empty
master_user['soft deletion time stamp'].value_counts()

Series([], Name: soft deletion time stamp, dtype: int64)

In [92]:
# block credential 226 => bpk, pegawai berhenti, pkwt habis kontrak, dll
master_user['block credential'].value_counts()

False    7585
True      226
Name: block credential, dtype: int64

In [93]:
# useful column only
master_user = master_user[['user principal name','state','title','department','office','licenses','block credential','when created','last password change time stamp']]

In [94]:
len(master_user)

7811

In [96]:
# user_activity_detail = pd.read_csv('365-report/user-detail/Office365ActiveUserDetail4_7_2022 2 33 39 PM.csv')
user_activity_detail = pd.read_csv('365-report/september/Office365ActiveUserDetail9_12_2022 6 10 51 AM.csv')

user_activity_detail.columns = map(str.lower, user_activity_detail.columns)

In [97]:
user_activity_detail.head()

,report refresh date,user principal name,display name,is deleted,deleted date,has exchange license,has onedrive license,has sharepoint license,has skype for business license,has yammer license,has teams license,exchange last activity date,onedrive last activity date,sharepoint last activity date,skype for business last activity date,yammer last activity date,teams last activity date,exchange license assign date,onedrive license assign date,sharepoint license assign date,skype for business license assign date,yammer license assign date,teams license assign date,assigned products
0,2022-09-08,uma_ak.i@bi.go.id,Uma Adi Kusuma (PKWT),False,NaN,False,True,True,True,False,True,NaN,2022-09-08,2022-09-07,NaN,NaN,2022-09-08,NaN,2022-06-29,2022-06-29,2022-06-29,NaN,2022-06-29,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E3
1,2022-09-08,andree@bi.go.id,Andree Breitner Makahinda,False,NaN,False,True,True,True,False,True,NaN,2021-12-08,2021-10-07,NaN,NaN,2022-01-26,NaN,2020-03-18,2020-03-18,2020-03-18,NaN,2020-03-18,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E3
2,2022-09-08,dwi_as@bi.go.id,Dwi Aris Setyawan,False,NaN,False,True,True,True,False,True,NaN,2022-09-08,2022-09-08,NaN,2021-12-03,2022-09-08,NaN,2020-10-09,2020-10-09,2020-10-09,NaN,2020-10-09,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E...
3,2022-09-08,indra_m@bi.go.id,Indra Maandri,False,NaN,False,True,True,True,False,True,NaN,2022-09-02,2022-09-07,NaN,NaN,2022-09-08,NaN,2020-10-05,2020-10-05,2020-10-05,NaN,2020-10-05,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E3
4,2022-09-08,e_handoko@bi.go.id,Eko Handoko,False,NaN,False,False,False,False,False,False,NaN,2022-01-31,2022-09-08,NaN,NaN,2022-08-30,NaN,NaN,NaN,NaN,NaN,NaN,MICROSOFT POWER APPS PLAN 2 TRIAL


In [98]:
user_activity_detail['is deleted'].value_counts()

False    6156
Name: is deleted, dtype: int64

In [99]:
len(user_activity_detail)

6156

In [100]:
# add column
user_activity_detail['has actity detail']=1

# Merge

In [101]:
data = master_user.merge(user_activity_detail, on='user principal name', how='left')
data.head()


,user principal name,state,title,department,office,licenses,block credential,when created,last password change time stamp,report refresh date,display name,is deleted,deleted date,has exchange license,has onedrive license,has sharepoint license,has skype for business license,has yammer license,has teams license,exchange last activity date,onedrive last activity date,sharepoint last activity date,skype for business last activity date,yammer last activity date,teams last activity date,exchange license assign date,onedrive license assign date,sharepoint license assign date,skype for business license assign date,yammer license assign date,teams license assign date,assigned products,has actity detail
0,ar_mubarok@bi.go.id,KPWDN,Pelaksana,JR.KANTOR PERWAKILAN BI JEMBER,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,Office 365 E3+Microsoft Power Automate Free+En...,False,2019-07-18 07:57:32Z,2022-06-26 23:45:44Z,2022-09-08,A Robith Mubarok,False,NaN,False,True,True,True,False,True,NaN,2022-07-16,2022-09-08,NaN,NaN,2022-09-08,NaN,2020-03-15,2020-03-15,2020-03-15,NaN,2020-03-15,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E...,1.0
1,amirullah@bi.go.id,KP,Manajer,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,Office 365 E3+Microsoft Power Automate Free+En...,False,2019-07-18 07:55:50Z,2022-06-29 00:13:10Z,2022-09-08,A. Amirullah Amir,False,NaN,False,True,True,True,False,True,NaN,2022-09-05,2022-09-07,NaN,NaN,2022-09-08,NaN,2020-03-15,2020-03-15,2020-03-15,NaN,2020-03-15,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E...,1.0
2,donanto@bi.go.id,KP,Direktur,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,KEPALA GRUP-DPSP.GRUP ELEKTRONIFIKASI & STANDA...,Office 365 E3+Microsoft Power Automate Free+En...,False,2019-07-18 07:55:29Z,2022-09-08 23:27:51Z,2022-09-08,A. Donanto H. W.,False,NaN,False,True,True,True,False,True,2021-09-13,2022-08-29,2022-09-08,NaN,2021-12-06,2022-09-08,NaN,2020-03-15,2020-03-15,2020-03-15,NaN,2020-03-15,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E...,1.0
3,arasta@bi.go.id,KP,Deputi Direktur,BINS.INSTITUT BI,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,Office 365 E3+Enterprise Mobility + Security E3,False,2019-07-18 07:54:50Z,2022-06-28 07:37:37Z,2022-09-08,A. Farid Aulia,False,NaN,False,True,True,True,False,True,NaN,2021-10-27,2022-08-24,NaN,2021-05-03,2022-08-01,NaN,2020-03-15,2020-03-15,2020-03-15,NaN,2020-03-15,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E3,1.0
4,afauzi_b@bi.go.id,KPWDN,Manajer,BD.KANTOR PERWAKILAN BI PROVINSI JAWA BARAT,KEPALA UNIT PUR-BD.SPMI.UNIT LAYANAN & PENGOLA...,Office 365 E3+Microsoft Power Automate Free+En...,False,2019-07-18 07:59:29Z,2022-06-15 09:58:57Z,2022-09-08,A. Fauzi Batubara,False,NaN,False,True,True,True,False,True,NaN,2022-09-06,2022-09-08,NaN,2021-06-29,2022-09-08,NaN,2020-03-15,2020-03-15,2020-03-15,NaN,2020-03-15,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E...,1.0


# Export

In [102]:
# complete
data.to_csv('merged-all-sep.csv', index=False)

In [103]:
# essentials
x = data[['user principal name','display name','state','title','department','office','block credential',
         'has onedrive license','onedrive last activity date',
         'has sharepoint license','sharepoint last activity date',
         'has teams license','teams last activity date',
        'report refresh date'
         ]]
x.to_csv('merged-essentials-sep.csv', index=False)